<a href="https://colab.research.google.com/github/akhileshkumbhar05-ui/computer-vision-course-project/blob/main/Code/Copy_of_Vehicle_Corridor_Traffic_Analytics_Edge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Project dirs
WORKDIR = "/content/drive/MyDrive/Computer Vision"
import pathlib, os
for d in [WORKDIR, f"{WORKDIR}/configs", f"{WORKDIR}/data", f"{WORKDIR}/experiments", f"{WORKDIR}/models", f"{WORKDIR}/outputs"]:
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)

# Core deps
!pip -q install "ultralytics==8.*" opencv-python numpy pandas matplotlib tqdm pyyaml
# For optional tracking metrics
!pip -q install motmetrics lapx filterpy onnx onnxruntime


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00


In [2]:
from pathlib import Path
import glob, os

BASE     = Path("/content/drive/MyDrive/Computer Vision/datasets/UA-DETRAC")
IMG_ROOT = Path("/content/drive/MyDrive/Computer Vision/datasets/UA-DETRAC/DETRAC-Images")
ANN_ROOT = Path("/content/drive/MyDrive/Computer Vision/datasets/UA-DETRAC/DETRAC-Train-Annotations-XML")

assert IMG_ROOT.exists(), f"Images alias not found: {IMG_ROOT}"
assert ANN_ROOT.exists(), f"Annotations alias not found: {ANN_ROOT}"

print("IMG_ROOT:", IMG_ROOT)
print("ANN_ROOT:", ANN_ROOT)
print("Sample MVI dirs:", sorted([d.name for d in IMG_ROOT.iterdir() if d.is_dir() and d.name.startswith("MVI_")])[:5])
print("#XML:", len(list(ANN_ROOT.glob("MVI_*.xml"))))


IMG_ROOT: /content/drive/MyDrive/Computer Vision/datasets/UA-DETRAC/DETRAC-Images
ANN_ROOT: /content/drive/MyDrive/Computer Vision/datasets/UA-DETRAC/DETRAC-Train-Annotations-XML
Sample MVI dirs: ['MVI_20011', 'MVI_20012', 'MVI_20032', 'MVI_20033', 'MVI_20034']
#XML: 60


In [ ]:
# # Converts UA-DETRAC annotations (XML) into YOLO boxes for 3 classes
# import cv2, glob, random, xml.etree.ElementTree as ET
# from pathlib import Path
# import os

# random.seed(42)
# YOLO_OUT = Path(f"{WORKDIR}/data/UA-DETRAC")
# for sub in ["images/train","images/val","labels/train","labels/val"]:
#     (YOLO_OUT/sub).mkdir(parents=True, exist_ok=True)

# CLS_MAP   = {"car":"car_suv","van":"car_suv","bus":"bus","truck":"truck","others":"truck"}
# YOLO_NAMES= ["car_suv","bus","truck"]
# NAME2ID   = {n:i for i,n in enumerate(YOLO_NAMES)}

# xml_files = sorted(glob.glob(str(ANN_ROOT/"MVI_*.xml")))
# random.shuffle(xml_files)
# val_set = set(xml_files[: max(5, len(xml_files)//10)])  # ~10%

# def voc_to_yolo(xmin,ymin,xmax,ymax,W,H):
#     cx=(xmin+xmax)/(2*W); cy=(ymin+ymax)/(2*H)
#     w=(xmax-xmin)/W; h=(ymax-ymin)/H
#     return cx,cy,w,h

# def convert_sequence(xml_path: str):
#     seq = Path(xml_path).stem
#     seq_dir = IMG_ROOT/seq
#     if not seq_dir.exists():
#         cands = list(IMG_ROOT.rglob(seq))
#         if not cands:
#             print(f"[WARN] frames for {seq} not found; skip."); return 0,0
#         seq_dir = cands[0]
#     frames = sorted(glob.glob(str(seq_dir/"*.jpg")) + glob.glob(str(seq_dir/"*.png")))
#     if not frames:
#         print(f"[WARN] no frames in {seq_dir}; skip."); return 0,0

#     root = ET.parse(xml_path).getroot()
#     per_frame = {}
#     for fr in root.findall(".//frame"):
#         num = int(fr.attrib.get("num","0"))
#         tlist = fr.find("target_list")
#         cur=[]
#         if tlist is not None:
#             for t in tlist.findall("target"):
#                 attr = t.find("attribute")
#                 vtype = (attr.attrib.get("vehicle_type","") if attr is not None else "").lower().strip()
#                 if vtype not in CLS_MAP: continue
#                 cid = NAME2ID[CLS_MAP[vtype]]
#                 box = t.find("box")
#                 if box is None: continue
#                 xmin=float(box.attrib["left"]); ymin=float(box.attrib["top"])
#                 w=float(box.attrib["width"]); h=float(box.attrib["height"])
#                 xmax=xmin+w; ymax=ymin+h
#                 cur.append((cid,xmin,ymin,xmax,ymax))
#         per_frame[num]=cur

#     split = "val" if xml_path in val_set else "train"
#     img_out = YOLO_OUT/"images"/split
#     lbl_out = YOLO_OUT/"labels"/split
#     copied=labeled=0

#     for i, fpath in enumerate(frames, start=1):
#         img = cv2.imread(fpath)
#         if img is None: continue
#         H,W = img.shape[:2]
#         out_img = img_out/f"{seq}_{i:06d}.jpg"
#         cv2.imwrite(str(out_img), img); copied += 1
#         ytxt = lbl_out/f"{seq}_{i:06d}.txt"
#         lines=[
#             f"{cid} {voc_to_yolo(xmin,ymin,xmax,ymax,W,H)[0]:.6f} {voc_to_yolo(xmin,ymin,xmax,ymax,W,H)[1]:.6f} "
#             f"{voc_to_yolo(xmin,ymin,xmax,ymax,W,H)[2]:.6f} {voc_to_yolo(xmin,ymin,xmax,ymax,W,H)[3]:.6f}"
#             for (cid,xmin,ymin,xmax,ymax) in per_frame.get(i, [])
#         ]
#         if lines:
#             with open(ytxt,"w") as f: f.write("\n".join(lines)); labeled += 1
#         else:
#             open(ytxt,"a").close()
#     return copied,labeled

# tot_c=tot_l=0
# for x in xml_files:
#     c,l = convert_sequence(x)
#     tot_c+=c; tot_l+=l

# print(f"✅ UA-DETRAC → YOLO done. Copied frames: {tot_c} | Labeled frames: {tot_l}")

# cfg = Path(f"{WORKDIR}/configs/uadetrac.yaml")
# cfg.write_text(f"""# auto-generated
# path: {YOLO_OUT.as_posix()}
# train: images/train
# val: images/val
# names: {YOLO_NAMES}
# """)
# print("Wrote", cfg)


KeyboardInterrupt: 

In [4]:
# FAST CONVERTER: make a tiny UA-DETRAC split directly on /content (SSD)
from pathlib import Path
import cv2, glob, random, re, xml.etree.ElementTree as ET
from tqdm import tqdm

# Use your existing vars (strings are fine); wrap as Path for safety
IMG_ROOT = Path(IMG_ROOT)
ANN_ROOT = Path(ANN_ROOT)
WORKDIR  = Path("/content/drive/MyDrive/Computer Vision")

# ---- how small? tweak here ----
MAX_SEQS_TRAIN = 6
MAX_SEQS_VAL   = 2
MAX_FRAMES_PER_SEQ = 200

DEST = WORKDIR / "data/UA-DETRAC_lite"
for sub in ["images/train","images/val","labels/train","labels/val"]:
    (DEST/sub).mkdir(parents=True, exist_ok=True)

def voc_to_yolo(xmin,ymin,xmax,ymax,W,H):
    cx=(xmin+xmax)/(2*W); cy=(ymin+ymax)/(2*H)
    w=(xmax-xmin)/W; h=(ymax-ymin)/H
    return cx,cy,w,h

def parse_xml(xml_path: Path):
    """Return dict: frame_idx -> list of (cid,xmin,ymin,xmax,ymax)"""
    CLS_MAP = {"car":"car_suv","van":"car_suv","bus":"bus","truck":"truck","others":"truck"}
    NAME2ID = {"car_suv":0,"bus":1,"truck":2}
    root = ET.parse(str(xml_path)).getroot()
    per_frame = {}
    for fr in root.findall(".//frame"):
        num = int(fr.attrib.get("num","0"))
        cur=[]
        tlist = fr.find("target_list")
        if tlist is not None:
            for t in tlist.findall("target"):
                attr = t.find("attribute")
                vtype = (attr.attrib.get("vehicle_type","") if attr is not None else "").lower().strip()
                if vtype not in CLS_MAP:
                    continue
                cid = NAME2ID[CLS_MAP[vtype]]
                box = t.find("box")
                if box is None:
                    continue
                xmin=float(box.attrib["left"]); ymin=float(box.attrib["top"])
                w=float(box.attrib["width"]); h=float(box.attrib["height"])
                xmax=xmin+w; ymax=ymin+h
                cur.append((cid,xmin,ymin,xmax,ymax))
        per_frame[num]=cur
    return per_frame

def convert_one(xml_path: Path, split: str):
    seq = xml_path.stem
    seq_dir = IMG_ROOT/seq
    if not seq_dir.exists():
        cands = list(IMG_ROOT.rglob(seq))
        if not cands:
            print(f"[WARN] frames for {seq} not found; skipping.");
            return 0,0
        seq_dir = cands[0]
    frames = sorted(glob.glob(str(seq_dir/"*.jpg")) + glob.glob(str(seq_dir/"*.png")))
    if not frames:
        print(f"[WARN] no frames in {seq_dir}; skipping.");
        return 0,0

    per_frame = parse_xml(xml_path)
    frames = frames[:MAX_FRAMES_PER_SEQ]  # keep only first K
    img_out = DEST/f"images/{split}"
    lbl_out = DEST/f"labels/{split}"

    copied=labeled=0
    for i, fpath in enumerate(tqdm(frames, desc=f"{seq} [{split}]"), start=1):
        img = cv2.imread(fpath)
        if img is None:
            continue
        H,W = img.shape[:2]
        out_img = img_out/f"{seq}_{i:06d}.jpg"
        cv2.imwrite(str(out_img), img); copied += 1
        ytxt = lbl_out/f"{seq}_{i:06d}.txt"
        lines=[]
        for (cid,xmin,ymin,xmax,ymax) in per_frame.get(i, []):
            cx,cy,w,h = voc_to_yolo(xmin,ymin,xmax,ymax,W,H)
            lines.append(f"{cid} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")
        if lines:
            with open(ytxt,"w") as f: f.write("\n".join(lines)); labeled += 1
        else:
            open(ytxt,"a").close()
    return copied,labeled

# pick a few sequences for train/val
xml_files = sorted(ANN_ROOT.glob("MVI_*.xml"))
random.seed(42)
random.shuffle(xml_files)
train_xmls = xml_files[:MAX_SEQS_TRAIN]
val_xmls   = xml_files[MAX_SEQS_TRAIN:MAX_SEQS_TRAIN+MAX_SEQS_VAL]

tot_c=tot_l=0
for x in train_xmls:
    c,l = convert_one(x, "train")
    tot_c+=c; tot_l+=l
for x in val_xmls:
    c,l = convert_one(x, "val")
    tot_c+=c; tot_l+=l

print(f"✅ Lite conversion done. Copied frames: {tot_c} | Labeled frames: {tot_l}")
yaml_path = WORKDIR/"configs/uadetrac_lite.yaml"
yaml_path.write_text(f"""# auto-generated lite split (direct)
path: {DEST.as_posix()}
train: images/train
val: images/val
names: ['car_suv','bus','truck']
""")
print("Wrote", yaml_path)


MVI_63562 [val]: 100%|██████████| 200/200 [00:10<00:00, 18.83it/s]

✅ Lite conversion done. Copied frames: 1600 | Labeled frames: 1600
Wrote /content/drive/MyDrive/Computer Vision/configs/uadetrac_lite.yaml


In [5]:
from ultralytics import YOLO

MODEL  = "yolov8n.pt"   # nano
IMGSZ  = 640            # 512–640 is fast; 960 is slower
EPOCHS = 5              # quick baseline
BATCH  = 16             # lower if OOM (8/4)

model = YOLO(MODEL)
results = model.train(
    data="/content/drive/MyDrive/Computer Vision/configs/uadetrac_lite.yaml",
    imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, device=0,
    cache=True, freeze=10, mosaic=0.0, mixup=0.0, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4,
    workers=2, optimizer="adamw", amp=True, val=False, save_period=0,
    project=f"{WORKDIR}/experiments", name="uad_lite_yolov8n"
)

# one validation at the end
best = f"{WORKDIR}/experiments/uad_lite_yolov8n/weights/best.pt"
metrics = YOLO(best).val(
    data="/content/drive/MyDrive/Computer Vision/configs/uadetrac_lite.yaml",
    imgsz=IMGSZ, project=f"{WORKDIR}/experiments", name="uad_lite_val",
    plots=False, save_json=False
)
print(metrics)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Computer Vision/configs/uadetrac_lite.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4, imgsz=640

In [ ]:
from ultralytics import YOLO
best = f"{WORKDIR}/experiments/uad_lite_yolov8n/weights/best.pt"
y = YOLO(best)
y.train(data="/content/evcv/configs/uadetrac_lite.yaml", imgsz=640, epochs=3, batch=16,
        device=0, cache=True, freeze=0, mosaic=0.0, mixup=0.0, amp=True,
        val=False, save_period=0, project=f"{WORKDIR}/experiments", name="uad_lite_yolov8n_tune")
YOLO(f"{WORKDIR}/experiments/uad_lite_yolov8n_tune/weights/best.pt").val(
    data="/content/evcv/configs/uadetrac_lite.yaml", imgsz=640,
    project=f"{WORKDIR}/experiments", name="uad_lite_val2", plots=False, save_json=False)


Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/evcv/configs/uadetrac_lite.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/evcv/experiments/uad_lite_yolov8n/weights/best.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=uad_lite_yolov8n_tune, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fce14418f50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
import pandas as pd, numpy as np, cv2, glob, os
from pathlib import Path
import motmetrics as mm
from ultralytics import YOLO

VAL_IMG_DIR = "/content/evcv/data/UA-DETRAC_lite/images/val"
val_imgs = sorted(glob.glob(f"{VAL_IMG_DIR}/*.jpg"))
subset = val_imgs[:400] if len(val_imgs) > 400 else val_imgs
listfile = Path(f"{WORKDIR}/experiments/val_list.txt")
listfile.write_text("\n".join(subset))

best = f"{WORKDIR}/experiments/uad_lite_yolov8n_tune/weights/best.pt"
if not os.path.exists(best):
    best = f"{WORKDIR}/experiments/uad_lite_yolov8n/weights/best.pt"
yolo_val = YOLO(best)
yolo_val.track(source=str(listfile), imgsz=640, tracker="bytetrack.yaml",
               conf=0.25, iou=0.5, persist=True, save=True,
               project=f"{WORKDIR}/experiments", name="uad_lite_track_bt")

track_labels_dir = sorted(glob.glob(f"{WORKDIR}/experiments/uad_lite_track_bt/*/tracks/labels"))[-1]
print("Track labels dir:", track_labels_dir)

def bbox_iou(a, b):
    ax1, ay1, aw, ah = a; bx1, by1, bw, bh = b
    ax2, ay2 = ax1+aw, ay1+ah; bx2, by2 = bx1+bw, by1+bh
    inter = max(0, min(ax2,bx2)-max(ax1,bx1)) * max(0, min(ay2,by2)-max(ay1,by1))
    ua = aw*ah + bw*bh - inter + 1e-9
    return inter/ua

def read_tracker_results(labels_dir: str):
    rows=[]
    for lf in sorted(glob.glob(f"{labels_dir}/frame_*.txt")):
        frame = int(Path(lf).stem.split("_")[-1])
        for line in open(lf):
            p = line.strip().split()
            if len(p) < 7: continue
            tid = int(float(p[0])); cx,cy,w,h = map(float, p[1:5]); conf=float(p[5])
            x = cx - w/2.0; y = cy - h/2.0
            rows.append([frame, tid, x, y, w, h, conf, -1, -1, -1])
    return pd.DataFrame(rows, columns=["FrameId","Id","X","Y","W","H","Conf","A","B","C"])

def read_gt_from_yolo(val_list_txt: str, labels_root: str):
    rows=[]
    with open(val_list_txt) as f:
        frames=[l.strip() for l in f if l.strip()]
    for i, img_path in enumerate(frames, start=1):
        stem = Path(img_path).stem
        lbl = Path("/content/evcv/data/UA-DETRAC_lite/labels/val")/f"{stem}.txt"
        img = cv2.imread(img_path)
        if img is None: continue
        H,W = img.shape[:2]
        if lbl.exists():
            for line in open(lbl):
                parts=line.strip().split()
                if len(parts)!=5: continue
                _, cx,cy,w,h = parts
                cx,cy,w,h = map(float,[cx,cy,w,h])
                x=(cx-w/2.0)*W; y=(cy-h/2.0)*H
                rows.append([i, -1, x, y, w*W, h*H, 1.0, -1,-1,-1])
    return pd.DataFrame(rows, columns=["FrameId","Id","X","Y","W","H","Conf","A","B","C"])

GT_MOT = read_gt_from_yolo(str(listfile), "/content/evcv/data/UA-DETRAC_lite/labels/val")
TR_MOT = read_tracker_results(track_labels_dir)

acc = mm.MOTAccumulator(auto_id=True)
for fr in sorted(GT_MOT["FrameId"].unique()):
    gt_boxes = GT_MOT[GT_MOT.FrameId==fr][["X","Y","W","H"]].values
    tr_boxes = TR_MOT[TR_MOT.FrameId==fr][["X","Y","W","H"]].values
    import numpy as np
    D = np.ones((len(gt_boxes), len(tr_boxes)))
    for i,a in enumerate(gt_boxes):
        for j,b in enumerate(tr_boxes):
            D[i,j] = 1 - bbox_iou(a,b)
    acc.update(list(range(len(gt_boxes))), list(range(len(tr_boxes))), D, frameid=fr)

mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['idf1','idp','idr','mota','motp','num_switches'], name='summary')
print(summary)
summary.to_csv(f"{WORKDIR}/outputs/motmetrics_summary.csv")
print("Saved:", f"{WORKDIR}/outputs/motmetrics_summary.csv")



image 1/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000001.jpg: 384x640 8 car_suvs, 1 truck, 50.0ms
image 2/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000002.jpg: 384x640 8 car_suvs, 1 truck, 6.4ms
image 3/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000003.jpg: 384x640 8 car_suvs, 1 truck, 6.0ms
image 4/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000004.jpg: 384x640 8 car_suvs, 7.2ms
image 5/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000005.jpg: 384x640 8 car_suvs, 5.8ms
image 6/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000006.jpg: 384x640 8 car_suvs, 6.6ms
image 7/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000007.jpg: 384x640 8 car_suvs, 6.0ms
image 8/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000008.jpg: 384x640 8 car_suvs, 6.5ms
image 9/400 /content/evcv/data/UA-DETRAC_lite/images/val/MVI_63553_000009.jpg: 384x640 8 car_suvs, 6.0ms
image 10/400 /content/evcv

IndexError: list index out of range

In [ ]:
import onnxruntime as ort, time, cv2, numpy as np, glob, os
from ultralytics import YOLO

best = f"{WORKDIR}/experiments/uad_lite_yolov8n_tune/weights/best.pt"
if not os.path.exists(best):
    best = f"{WORKDIR}/experiments/uad_lite_yolov8n/weights/best.pt"

onnx_dir = f"{WORKDIR}/models/uad_lite_yolov8n_onnx"
YOLO(best).export(format="onnx", opset=12, dynamic=True, simplify=True, imgsz=640,
                  project=f"{WORKDIR}/models", name="uad_lite_yolov8n_onnx")
onnx_path = sorted(glob.glob(f"{onnx_dir}/*.onnx"))[0]

sess = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])
inp = sess.get_inputs()[0].name

val_imgs = sorted(glob.glob("/content/evcv/data/UA-DETRAC_lite/images/val/*.jpg"))
if val_imgs:
    im = cv2.imread(val_imgs[0])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (640, 640)).astype(np.float32)/255.0
    im = np.transpose(im, (2,0,1))[None,...]
    for _ in range(3): _ = sess.run(None, {inp: im})
    N=20; t0=time.time()
    for _ in range(N): _ = sess.run(None, {inp: im})
    dt=(time.time()-t0)/N
    print(f"[ONNX CPU] ~{1/dt:.1f} FPS ({dt*1000:.1f} ms) @ 640x640")
else:
    print("No val images to test latency.")


Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/evcv/experiments/uad_lite_yolov8n_tune/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 12.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.71...
ONNX: export success ✅ 15.8s, saved as '/content/evcv/experiments/uad_lite_yolov8n_tune/weights/best.onnx' (11.8 MB)

Export complete (16.7s)
Results saved to /content/evcv/experiments/ua

IndexError: list index out of range